# Point spread function covariance

In order to use the NLL with a MVG distribution, the assumption was that the covariance $\Sigma$ of the model was Identity in its representation, essentially for $\Sigma\in\mathbb{R}^{22500x22500}$, the matrix was represented as $\text{diag(1,1,...,1)}$

This assumption works for data (pixels) that are i.i.d, however in Radio astoronomy, this is not the case. Data are read in as fourier components, and during processing are convolved, this leads to *point spread*, where point sources have been smeared across multiple pixels. We know from [the webiste](https://sundog.stsci.edu/first/catalogs/readme.html) that VLA-FIRST, where RGZ108k (dataset used is from) has a point spread of 1.8 arcseeconds. This means that each pixel represents a distance $\mathbf{x}$ of $(\text{RA},\delta)=(1.8,1.8)$. We also know the point-spread in the north,south is 5.4,6.4.

## Assumptions:

- We will do away with the 6.4, and assume the point-spread distribution is governed only by the 5.4, which gives the point spread $p\sim \mathcal{N}(0,\sigma^2)$ wheere $\sigma=\frac{5.4}{2\sqrt{2\ln{2}}}$
- While calculations of distrance in $(\text{RA},\delta)$ can be done, for ease the calculation will be done for demonstration in Euclidean space $(x,y)$, allowing to take the $L_2$ norm.

The algorithm can be thought of as, for two pixels in an image $(i,j)$,$(k,l)$:
$$
d_{(i,j),(k,l)} = \sqrt{(i-k)^2+(j-l)^2}
$$

This results in a correlation:

$$
C_{(i,j),(k,l)} = \mathcal{N}(d;0,\sigma^2)
$$

The implementation is followed:

In [3]:
import numpy as np
from scipy.stats import norm

image_size = 150
sigma = 5.4 / (2 * np.sqrt(2 * np.log(2)))
normal = norm(loc=0,scale=sigma)

x,y = np.meshgrid(np.arange(image_size),np.arange(image_size),indexing = "ij")
pixel_coords = np.column_stack((x.ravel(),y.ravel()))

pixels = image_size**2
correlation_matrix = np.zeros((pixels,pixels))

In [ ]:
for q in range(pixels):
    i,j = pixel_coords[q]
    for r in range(pixels):
        if q == r: # Diagonal entries
            correlation_matrix[q,r]=1
        else:
            k,l = pixel_coords[r] # For pixel 2
            d =  d = np.sqrt((i - k) ** 2 + (j - l) ** 2)
            C = normal.pdf(d)
            correlation_matrix[q,r] = C